<a href="https://colab.research.google.com/github/Tstrebe2/umich-mads-capstone-project/blob/main/tim-yolo-v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

!{sys.executable} -m pip install pydicom -q
!{sys.executable} -mpip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt -q  # Yolo Dependencies

     |████████████████████████████████| 2.0 MB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 5.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path = '/content/drive/MyDrive/capstone/assets/stage_2_train_images.zip'

!unzip -qq {path}

In [8]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
import torch
import torchvision
from torch.profiler import profile, record_function, ProfilerActivity

import pydicom as dicom
from PIL import Image

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import copy

In [104]:
np.expand_dims(np.array([[1, 2], [1, 2]]), axis=0).shape

(1, 2, 2)

In [109]:
class CustomImageDataset(Dataset):
  def __init__(self, annotations_file, img_dir, indices=None, transform=None, target_transform=None):
    if indices:
      self.img_labels = pd.read_csv(annotations_file).iloc[indices].fillna(0)
    else:
      self.img_labels = pd.read_csv(annotations_file).fillna(0)

    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = ''.join([os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]), '.dcm'])
    
    image = dicom.dcmread(img_path)
    image = image.pixel_array
    # image = Image.fromarray(image.pixel_array)#.convert(mode='RGB')

    target = self.img_labels.iloc[idx, 1:-1].values.astype(np.float32)

    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        target = self.target_transform(target)
    return image, target

  def get_classes(self):
    return self.img_labels.iloc[:, -1].unique()

  classes = property(get_classes)

In [110]:
annotations_file = '/content/drive/MyDrive/capstone/assets/stage_2_train_labels.csv.zip'
img_dir = '/content'

label_df = pd.read_csv(annotations_file)

X_train, X_test = train_test_split(label_df, test_size=.2, stratify=label_df.Target, random_state=99)
X_val, X_test, = train_test_split(X_test, test_size=.5, stratify=X_test.Target, random_state=99)
X_train, X_val, X_test = X_train.index.tolist(), X_val.index.tolist(), (X_test.index.tolist())

len(X_train), len(X_val),  len(X_test)

(24181, 3023, 3023)

In [111]:
mean = 0.46199
std = 0.23566

transforms = dict(
  train = torchvision.transforms.Compose([
      # torchvision.transforms.Resize(256),
      # torchvision.transforms.CenterCrop(224),
      # torchvision.transforms.RandomHorizontalFlip(.2),
      torchvision.transforms.ToTensor(),
      torchvision.transforms.Normalize(mean, std),
  ]),
  val = torchvision.transforms.Compose([
      # torchvision.transforms.Resize(256),
      # torchvision.transforms.CenterCrop(224),
      torchvision.transforms.ToTensor(),
      torchvision.transforms.Normalize(mean, std),
  ])
)

datasets = dict(
    train=CustomImageDataset(annotations_file, img_dir, indices=X_train, transform=transforms['train'], target_transform=torch.from_numpy),
    val=CustomImageDataset(annotations_file, img_dir, indices=X_val, transform=transforms['val'], target_transform=torch.from_numpy),
    test=CustomImageDataset(annotations_file, img_dir, indices=X_test, transform=transforms['val'], target_transform=torch.from_numpy),
)

In [112]:
batch_size = 32

data_loaders = dict(
    train=DataLoader(datasets['train'], batch_size=batch_size, shuffle=True),
    val=DataLoader(datasets['val'], batch_size=batch_size, shuffle=True),
    test=DataLoader(datasets['test'], batch_size=batch_size),
)

dataset_sizes = {x:len(datasets[x]) for x in ('train', 'val', 'test')}

print('training steps per epoch:', dataset_sizes['train']//batch_size)
print('validation steps per epoch:', dataset_sizes['val']//batch_size)
print('test steps per epoch:', dataset_sizes['test']//batch_size)

training steps per epoch: 755
validation steps per epoch: 94
test steps per epoch: 94


In [96]:
# Uncomment and run to get mean and standard deviation
# for x, y in data_loaders['train']:
#   break

# x.mean(), x.std()

(tensor(0.48354), tensor(0.25067))

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, channels=1)
model = model.cuda()

In [120]:
optimizer = torch.optim.SGD(g[2], lr=1e-3, momentum=0.9, nesterov=True)
loss = torch.nn.

for (inputs, targets) in data_loaders['train']:
  inputs = inputs.cuda()
  targets = targets.cuda()
  break

with torch.set_grad_enabled(True):
  model.train()
  optimizer.zero_grad()
  outputs = model(inputs)

outputs.shape

torch.Size([32, 64512, 85])